<a href="https://colab.research.google.com/github/aljeshishe/notebooks/blob/master/vgg_gpu_v2_with_wb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 
import sys 

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from time import time 

In [0]:
def load_dataset():
    #load dataset
    (trainX, trainY),(testX, testY) = cifar10.load_data()
    #one hot encode the target 
    trainY = keras.utils.to_categorical(trainY)
    testY = keras.utils.to_categorical(testY)
    return trainX, trainY, testX, testY

def validation_split(testX, testY, valid_X, valid_Y, v_split):
    
    index_of_validation = int(v_split * len(testX))
    valid_X.extend(testX[-index_of_validation:])
    valid_Y.extend(testY[-index_of_validation:])
    testX = testX[:-index_of_validation]
    testY = testY[:-index_of_validation]
    return testX, testY, np.asarray(valid_X), np.asarray(valid_Y)

def normalize(train,test,valid):
    # convert from integers to float 
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    valid_norm = valid.astype('float32')
    #normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    valid_norm = valid_norm / 255.0
    return train_norm, test_norm,valid_norm 

# plot diagnostic learning curves
def summarize_diagnostics(history):
    plt.subplots(figsize = (7,7))
    # plot loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')

    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    plt.show()
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()



In [0]:
# load dataset
trainX, trainY, testX, testY = load_dataset()
#get validation set 
valid_X = []
valid_Y = []
testX, testY, validX, validY = validation_split(testX, testY, valid_X, valid_Y,v_split=0.5)

# normalize the data
trainX, testX,validX = normalize(trainX, testX,validX)


In [0]:
import os
# use gpu/cpu/tpu
# see details in https://colab.research.google.com/drive/1cpuwjKTJbMjlvZ7opyrWzMXF_NYnjkiE#scrollTo=y3gk7nSvTUFZ
gpus = tf.config.experimental.list_physical_devices('GPU')
COLAB_TPU_ADDR = os.environ.get('COLAB_TPU_ADDR')
if COLAB_TPU_ADDR:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + COLAB_TPU_ADDR)
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  print('Running on TPU ')  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
globals()

{'BatchNormalization': tensorflow.python.keras.layers.normalization_v2.BatchNormalization,
 'COLAB_TPU_ADDR': None,
 'Conv2D': tensorflow.python.keras.layers.convolutional.Conv2D,
 'Dense': tensorflow.python.keras.layers.core.Dense,
 'Dropout': tensorflow.python.keras.layers.core.Dropout,
 'Flatten': tensorflow.python.keras.layers.core.Flatten,
 'ImageDataGenerator': tensorflow.python.keras.preprocessing.image.ImageDataGenerator,
 'In': ['',
  "import tensorflow as tf\n\nmodel = tf.keras.models.load_model('/content/wandb/run-20200429_181146-3lftvc8y/model-best.h5')",
  'import tensorflow as tf \nfrom tensorflow import keras\nimport numpy as np \nimport matplotlib.pyplot as plt \nimport sys \n\nfrom tensorflow.keras.datasets import cifar10\nfrom tensorflow.keras.models import Sequential \nfrom tensorflow.keras.layers import Dense, Dropout, Flatten \nfrom tensorflow.keras.layers import Conv2D, MaxPooling2D\nfrom tensorflow.keras.optimizers import SGD\nfrom tensorflow.keras import regular

In [0]:
import os
os.mkdir('/content/drive/My Drive/Colab Notebooks')


In [0]:
pip install wandb -q

In [0]:
import wandb
from wandb.keras import WandbCallback
config = dict(
  batch_size = 64,          # input batch size for training (default: 64)
  epochs = 1000,             # number of epochs to train (default: 10)
  lr = 0.001,               # learning rate (default: 0.01)
  momentum = 0.9,          # SGD momentum (default: 0.5) 
  seed = 42,               # random seed (default: 42)
  log_interval = 10,     # how many batches to wait before logging training status
  weight_decay = 0.0005
)
wandb.init(config=config)
config = wandb.config

In [0]:
def vgg():
  input_shape = (trainX.shape[1], trainX.shape[2], 3)
  
  # Define the model architecture - This is a simplified version of the VGG19 architecture
  model = Sequential()
  
  # Set of Conv2D, Conv2D, MaxPooling2D layers with 32 and 64 filters
  model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.3))

  model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 128 filters
  model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 256 filters
  model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 512 filters
  model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 512, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Another set of Conv2D, Conv2D, MaxPooling2D layers with 512 filters
  model.add(Conv2D(filters = 1024, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Conv2D(filters = 1024, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(2, 2)))

  # Flatten
  model.add(Flatten())
  model.add(Dense(512, activation ='relu', kernel_regularizer=regularizers.l2(config.weight_decay)))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  num_classes = 10
  model.add(Dense(num_classes, activation = "softmax"))
  opt = SGD(lr = config.lr, momentum=config.momentum)
  model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [0]:
# define model
def baseline():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same', input_shape = (32,32,3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.4))
   

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = 'softmax'))
    
    #compile model 
    opt = SGD(lr = config.lr, momentum=config.momentum)
    model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model


In [0]:
with strategy.scope():
  model = vgg()
#epochs = 400
#create data generator 
datagen = ImageDataGenerator(width_shift_range = 0.1, height_shift_range = 0.1, horizontal_flip = True)
#iterator 
train = datagen.flow(trainX, trainY, batch_size=config.batch_size)
# fit model
steps = int(trainX.shape[0]/ 64)
history = model.fit_generator(train, steps_per_epoch=steps, epochs=config.epochs, validation_data=(validX, validY), verbose=1,
                              callbacks=[WandbCallback(validation_data=(validX, validY),
                                                       save_model=True, verbose=1)])

wandb.save("mymodel.h5")
model.save(os.path.join(wandb.run.dir, "mymodel.h5"))

In [1]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/wandb/run-20200429_181146-3lftvc8y/model-best.h5')

OSError: ignored

In [13]:
model.predict(valid_X[0])

ValueError: ignored

In [5]:
model.layers

In [2]:
trainX[0].shape

NameError: ignored

In [0]:
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                 outputs=model.layers[0].output)
intermediate_output = intermediate_layer_model.predict(validX)

In [11]:
intermediate_output.shape

(5000, 32, 32, 32)

In [12]:
model.layers

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 128)      

In [0]:
# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))


In [0]:

summarize_diagnostics(history)


In [0]:
wandb.save("mymodel.h5")
model.save(os.path.join(wandb.run.dir, "mymodel.h5"))

In [0]:

import numpy as np
import os
import time

#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
import keras.applications as kapp
from keras.datasets import cifar10
def f():
  (x_train, y_train_cats), (x_test, y_test_cats) = cifar10.load_data()
  batch_size = 80
  x_train = x_train[:batch_size]
  x_train = np.repeat(np.repeat(x_train, 7, axis=1), 7, axis=2)
  model = kapp.VGG19()
  model.compile(optimizer='sgd', loss='categorical_crossentropy',
                metrics=['accuracy'])

  print("Running initial batch (compiling tile program)")
  y = model.predict(x=x_train, batch_size=batch_size)

  # Now start the clock and run 10 batches
  print("Timing inference...")
  start = time.time()
  for i in range(1):
      y = model.predict(x=x_train, batch_size=batch_size)
  print("Ran in {} seconds".format(time.time() - start))
f()

In [2]:
!nvidia-smi

Wed Apr 29 20:45:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  